# 🍽️ iFood — Intelligent Segmentation and Campaign Response Analysis



## Overview

This project analyzes the **effectiveness of promotional campaigns in a premium retail context at iFood**, using an **adapted dataset** for analytical purposes. The scenario involves multiple campaigns with distinct strategies and a rich dataset containing **demographic, behavioral, and consumption** information.

The goal is to turn heterogeneous campaign results into **reliable business decisions**, supporting the evolution of a data-driven marketing strategy focused on performance, efficiency, and risk reduction.



## Project Objectives

- Evaluate **customer response** to different promotional strategies  
- Compare campaign performance through **controlled (A/B) tests**  
- Define criteria for **evidence-based decision making**  
- Support campaign planning with a focus on **impact and return on investment**



## Analytical Approach

The project includes the following steps:

- Exploration and preparation of customer data  
- Analysis of customer profiles, purchasing behavior, and campaign history  
- Comparison of test groups and performance evaluation across approaches  
- Definition of metrics, hypotheses, and decision criteria  
- Interpretation of results with a focus on **business implications**



## Data

The dataset includes variables such as **income, household composition, recency, spending by category, purchase channels, website visits, campaign acceptance history**, as well as demographic and customer relationship attributes. It also includes the identification of **test groups** for experimentation.


## Expected Outcome

To build a **clear and reproducible framework** for evaluating campaigns with analytical rigor and turning data into **actionable recommendations**, supporting marketing decisions focused on **performance, risk, and investment efficiency**.

---


## 🔧 Setup and Import
Initial environment setup

In [11]:
import pandas as pd
import numpy as np

In [12]:
pd.set_option('display.max_columns', None)   # configure pandas to show ALL columns in display
pd.set_option('display.width', None)         # prevent awkward line wrapping
pd.set_option('display.max_colwidth', None)  # do not truncate column names/text

## Load dataset

In [19]:
# Load raw dataset
df = pd.read_excel("../data/ifood_raw.xlsx")

# Quick sanity check
display(df.tail())

,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp1,AcceptedCmp2,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,Complain,Z_CostContact,Z_Revenue,Response,Age,Customer_Days,marital_Divorced,marital_Married,marital_Single,marital_Together,marital_Widow,education_2n Cycle,education_Basic,education_Graduation,education_Master,education_PhD,MntTotal,MntRegularProds,AcceptedCmpOverall,Grupo
2200,61223,0,1,46,709,43,182,42,118,247,2,9,3,4,5,0,0,0,0,0,0,3,11,0,53,2540,0,1,0,0,0,0,0,1,0,0,1094,847,0,A
2201,64014,2,1,56,406,0,30,0,0,8,7,8,2,5,7,1,0,0,0,0,0,3,11,0,74,2178,0,0,0,1,0,0,0,0,0,1,436,428,1,B
2202,56981,0,0,91,908,48,217,32,12,24,1,2,3,13,6,0,0,0,1,0,0,3,11,0,39,2314,1,0,0,0,0,0,0,1,0,0,1217,1193,1,B
2203,69245,0,1,8,428,30,214,80,30,61,2,6,5,10,3,0,0,0,0,0,0,3,11,0,64,2315,0,0,0,1,0,0,0,0,1,0,782,721,0,B
2204,52869,1,1,40,84,3,61,2,1,21,3,3,1,4,7,0,0,0,0,0,0,3,11,1,66,2781,0,1,0,0,0,0,0,0,0,1,151,130,0,A
